# Implementing a Key-Value Database

## Introduction

In this guided project, we will be using the b-tree data structure from before as the building block for a fully functioning, saving-to-disk, key-value store.

A [key-value store](https://en.wikipedia.org/wiki/Key-value_database) is a database that operates similar to a Python dictionary but stored in a file. There are the important get and set methods that the dictionary contains, but the key-value store will also provide an API for saving to disk, loading from disk, range queries of our data, and others. Our goal will be to create an easy to use, flexible, and adaptable key value store that other developers could use in their projects.

For this guided project, we will inherit the majority of the behavior from the BTree class. However, we will redesign some of the API to make it more user friendly. Finally, we will be implementing some additional behavior to make it function as the key-value database that we expect.

## Importing the existing classes

We will import the "**NodeKey**", "**Node**" and "**BTree**" classes from the external python module called **btree.py**:

In [1]:
from btree import NodeKey, Node, BTree
#import pickle to dump and load
import pickle

## Creating a the class DQKV

### Adding the set() and get() methods

We will now create a class called **DQKV** (DataQuest Key-Value Store) inheriting properties from **BTree** class. Aside from all the existing methods, we will implement the **set()** and **get()** taking into account:
-  **set()**: The kv store will set a value for some given key only if value is not None. The set() method should require the uniqueness of the key: that means if the key exists in the kv store, then throw a duplicate error.
- **get()**: The kv store will get the value for the given key. However, if that key does not exist, the key-value store should throw an exception.

We should think about the API design as we develop the get() and set() methods. A handy class that we might want to use is the **NodeKey** class that we have implemented before. With this class, we can try to make it so the user of the kv store does not have to create a NodeKey objects first before calling set() on the kv store.

In [2]:
class DQKV(BTree):
    def set(self, key, value=None):
        if value is None:
            raise ValueError("A value must be introduced!")
        if self.search(self.root, key) is not None:
            raise KeyError("Key {} already exists!".format(key))
        kv_node = NodeKey(key, value)
        self.insert(kv_node)
        
    def get(self, key):
        kv_value = self.search(self.root, key)
        if kv_value is None:
            raise KeyError("Key {} does not exist!".format(key))
        return kv_value

In [3]:
#Test
dq = DQKV(4)
l1 = (i for i in range(100))
l2 = (i*1000 for i in range(100))
for key, value in zip(l1,l2):
    dq.set(key,value)
    
dq.get(50)

50000

### Overriding the initializer

Right now we allow any type to be set as our key. For our kv store to work with a b-tree, we need to set some data type as our key. If we don't then we would be doing comparisons on _ints_ and _strs_ for example.

We should override the initializer and set some type that we use to validate the inserted keys in our set() method.

An important thing to note is that when overriding the \__init__() method, we also want to also call the parent \__init__() method. If you don't then we would not be able to set the **self.t** and **self.root** properties that are expected.

In [4]:
class DQKV(BTree):
    def __init__(self, key_type):
        self.key_type = key_type
        #We automatically set the degree value for the BTree initialization, as 6
        super().__init__(6)
        
    def set(self, key, value=None):
        if not isinstance(key, self.key_type):
            raise KeyError("Key {} does not have type {}!".format(key, self.key_type))
        if self.search(self.root, key) is not None:
            raise KeyError("Key {} already exists!".format(key))
        if value is None:
            raise ValueError("A value must be introduced!")
        kv_node = NodeKey(key, value)
        self.insert(kv_node)
        
    def get(self, key):
        kv_value = self.search(self.root, key)
        if kv_value is None:
            raise KeyError("Key {} does not exist!".format(key))
        return kv_value

In [5]:
#Test
dq = DQKV(int)
l1 = (i for i in range(100))
l2 = (i*1000 for i in range(100))
for key, value in zip(l1,l2):
    dq.set(key,value)
    
dq.get(50)

50000

### Reimplementing the Range Queries

The current range queries for **greater_than()** and **less_than()** are a bit tedious to work with. First, you have to pass in the root node, then you have to pass in a value, an optional upper bound, and then an inclusive! All these can be frustrating for a user who just wants to do a simple range query on their data.

To solve this user experience problem, we will implement a new range query method that will abstract away the tedious work for the user. We will still use the old range queries under the hood, but we provide the user with better error returns so they can iteratively learn how to use the range query:

``` python
# This should return all the keys that are within the
# interval (0, 5) inclusive.
dqkv.range_query([0, 5], inclusive=True)
# This should return all values greater than 6.
dqkv.range_query([6, None])
# This should return all values less than 6.
dqkv.range_query([None, 6])
```

In [6]:
class DQKV(BTree):
    def __init__(self, key_type):
        self.key_type = key_type
        #We automatically set the degree value for the BTree initialization, as 6
        super().__init__(6)
        
    def set(self, key, value=None):
        if not isinstance(key, self.key_type):
            raise KeyError("Key {} does not have type {}!".format(key, self.key_type))
        if self.search(self.root, key) is not None:
            raise KeyError("Key {} already exists!".format(key))
        if value is None:
            raise ValueError("A value must be introduced!")
        kv_node = NodeKey(key, value)
        self.insert(kv_node)
        
    def get(self, key):
        kv_value = self.search(self.root, key)
        if kv_value is None:
            raise KeyError("Key {} does not exist!".format(key))
        return kv_value
    
    def range_query(self, q_range, inclusive=False):
        if len(q_range) != 2:
            raise ValueError("The range must have only 2 elements: minimum and maximum")
        low_range = q_range[0]
        high_range = q_range[1]
        if low_range is not None and high_range is not None:
            if inclusive and low_range > high_range:
                raise ValueError("When including the range thresholds, the minimum must be lower or equal than the maximum")
            if not inclusive and low_range >= high_range:
                raise ValueError("When not including the range thresholds, the minimum must be lower than the maximum")
            return self.greater_than(self.root, low_range, upper_bound=high_range, inclusive=inclusive)
        if low_range is not None:
            return self.greater_than(self.root, low_range, inclusive=inclusive)
        if high_range is not None:
            return self.less_than(self.root, high_range, inclusive=inclusive)

In [7]:
#Test
dq = DQKV(int)
l1 = (i for i in range(100))
l2 = (i*1000 for i in range(100))
for key, value in zip(l1,l2):
    dq.set(key,value)
    
print(dq.range_query([1,5]))
print(dq.range_query([1,5], inclusive=True))
print(dq.range_query([None,5]))
print(dq.range_query([95,None], inclusive=True))

[<NodeKey: (2, 2000)>, <NodeKey: (3, 3000)>, <NodeKey: (4, 4000)>]
[<NodeKey: (5, 5000)>, <NodeKey: (1, 1000)>, <NodeKey: (2, 2000)>, <NodeKey: (3, 3000)>, <NodeKey: (4, 4000)>]
[<NodeKey: (0, 0)>, <NodeKey: (1, 1000)>, <NodeKey: (2, 2000)>, <NodeKey: (3, 3000)>, <NodeKey: (4, 4000)>]
[<NodeKey: (95, 95000)>, <NodeKey: (96, 96000)>, <NodeKey: (97, 97000)>, <NodeKey: (98, 98000)>, <NodeKey: (99, 99000)>]


### Dumping and Loading the KV Store

The behavior we want to provide is to let a user pass in a filename to load or save to the object to disk. To make it easier for the user, we will attach a our filetype name to the end and check if that file is available to read or write out to.

However, one thing to keep in mind is that we want to provide a **load() function** that is not an instance method of the DQKV class. The reason being is that we want to load from a file to create a DQKV instance. If it was an instance method, then it would be defeating the purpose of the behavior it is trying to provide.

This is the behavior we would like to see:
```python
# Dumps out to a file called 'sample_kv.dqdb'
dqkv.dump('sample_kv')
# Loads from a file called 'sample_kv.dqdb' and
# assigns the loaded object to an instance using
# a function and not a instance method.
dqkv_loaded = load_dqkv('sample_kv')
```

In [8]:
def load_dqkv(filename):
    with open(filename + ".dqkv","rb") as f:
            return pickle.load(f)

In [9]:
class DQKV(BTree):
    def __init__(self, key_type):
        self.key_type = key_type
        #We automatically set the degree value for the BTree initialization, as 6
        super().__init__(6)
        
    def set(self, key, value=None):
        if not isinstance(key, self.key_type):
            raise KeyError("Key {} does not have type {}!".format(key, self.key_type))
        if self.search(self.root, key) is not None:
            raise KeyError("Key {} already exists!".format(key))
        if value is None:
            raise ValueError("A value must be introduced!")
        kv_node = NodeKey(key, value)
        self.insert(kv_node)
        
    def get(self, key):
        kv_value = self.search(self.root, key)
        if kv_value is None:
            raise KeyError("Key {} does not exist!".format(key))
        return kv_value
    
    def range_query(self, q_range, inclusive=False):
        if len(q_range) != 2:
            raise ValueError("The range must have only 2 elements: minimum and maximum")
        low_range = q_range[0]
        high_range = q_range[1]
        if low_range is not None and high_range is not None:
            if inclusive and low_range > high_range:
                raise ValueError("When including the range thresholds, the minimum must be lower or equal than the maximum")
            if not inclusive and low_range >= high_range:
                raise ValueError("When not including the range thresholds, the minimum must be lower than the maximum")
            return self.greater_than(self.root, low_range, upper_bound=high_range, inclusive=inclusive)
        if low_range is not None:
            return self.greater_than(self.root, low_range, inclusive=inclusive)
        if high_range is not None:
            return self.less_than(self.root, high_range, inclusive=inclusive)
        
    def dump(self,filename):
        with open(filename + ".dqkv","wb") as f:
            pickle.dump(self, f)
            return True
        return False

In [10]:
#Test
dq = DQKV(int)
l1 = (i for i in range(100))
l2 = (i*1000 for i in range(100))
for key, value in zip(l1,l2):
    dq.set(key,value)

dq.dump("dqkv_test")
new_dq = load_dqkv("dqkv_test")

print(new_dq.range_query([1,5]))

[<NodeKey: (2, 2000)>, <NodeKey: (3, 3000)>, <NodeKey: (4, 4000)>]


### Loading from Dictionary

We are going to write a new method, called **load_from_dict()**, that loads in keys and values from a Python dict.

We must take into account that a dict can have any key and we place a restriction on our keys.

In [11]:
class DQKV(BTree):
    def __init__(self, key_type):
        self.key_type = key_type
        #We automatically set the degree value for the BTree initialization, as 6
        super().__init__(6)
        
    def set(self, key, value=None):
        if not isinstance(key, self.key_type):
            raise KeyError("Key {} does not have type {}!".format(key, self.key_type))
        if self.search(self.root, key) is not None:
            raise KeyError("Key {} already exists!".format(key))
        if value is None:
            raise ValueError("A value must be introduced!")
        kv_node = NodeKey(key, value)
        self.insert(kv_node)
        
    def get(self, key):
        kv_value = self.search(self.root, key)
        if kv_value is None:
            raise KeyError("Key {} does not exist!".format(key))
        return kv_value
    
    def range_query(self, q_range, inclusive=False):
        if len(q_range) != 2:
            raise ValueError("The range must have only 2 elements: minimum and maximum")
        low_range = q_range[0]
        high_range = q_range[1]
        if low_range is not None and high_range is not None:
            if inclusive and low_range > high_range:
                raise ValueError("When including the range thresholds, the minimum must be lower or equal than the maximum")
            if not inclusive and low_range >= high_range:
                raise ValueError("When not including the range thresholds, the minimum must be lower than the maximum")
            return self.greater_than(self.root, low_range, upper_bound=high_range, inclusive=inclusive)
        if low_range is not None:
            return self.greater_than(self.root, low_range, inclusive=inclusive)
        if high_range is not None:
            return self.less_than(self.root, high_range, inclusive=inclusive)
        
    def dump(self,filename):
        with open(filename + ".dqkv","wb") as f:
            pickle.dump(self, f)
            return True
        return False
    
    def load_from_dict(self, dictionary):
        for key, value in dictionary.items():
            if isinstance(key, self.key_type):
                self.set(key, value)
            else:
                raise KeyError("Key {} does not have type {}!".format(key, self.key_type))

In [12]:
#Test
dq = DQKV(int)
l1 = (i for i in range(100))
l2 = (i*1000 for i in range(100))
for key, value in zip(l1,l2):
    dq.set(key,value)

new_keys = {
    150: 150000,
    151: 151000,
    152: 152000
}

dq.load_from_dict(new_keys)
print(dq.range_query([98,None]))

[<NodeKey: (99, 99000)>, <NodeKey: (150, 150000)>, <NodeKey: (151, 151000)>, <NodeKey: (152, 152000)>]
